### Cria tabela artigos_dados_xml para conter os dados extraídos dos arquivos .xml de cada artigo selecionado

### http://docs.scielo.org/projects/scielo-publishing-schema/pt_BR/1.6-branch/

In [1]:
local_dir = '/home/neilor/DADOS_SCIELO/alacip/'
xml_dir = local_dir+'artigos_xml/'
import scielo_mod as smod

In [2]:
import time
import os
import pandas as pd
import psycopg2 as pg
from pathlib import Path
from lxml import etree
from xml.etree.ElementTree import ParseError
import scielo_xml as sci
from bs4 import BeautifulSoup

In [3]:
query_create_table_artigos_dados_xml = """
drop table if exists alacip.artigos_dados_xml;
create table alacip.artigos_dados_xml
(
    scielo_id varchar,
    titulo text,
    resumo text,
    palavra_chave1 varchar,
    palavra_chave2 varchar,
    palavra_chave3 varchar,
    palavra_chave4 varchar,
    palavra_chave5 varchar,
    empirico varchar,
    tem_tabela varchar,
    tem_quadro varchar,
    tem_grafico varchar,
    tem_diagrama varchar,
    tem_figura varchar,
    abstract text,
    key_word1 varchar,
    key_word2 varchar,
    key_word3 varchar,
    key_word4 varchar,
    key_word5 varchar,
    texto text
);

CREATE INDEX texto_pt_idx ON alacip.artigos_dados_xml USING GIN (to_tsvector('portuguese', texto));

CREATE INDEX texto_en_idx ON alacip.artigos_dados_xml USING GIN (to_tsvector('english', texto));
"""

smod.execute_query(query_create_table_artigos_dados_xml)


### Gera comando SQL para inclusão dos dados do arquivo .xml na tabela dados_xml

In [6]:
import scielo_xml as xml
import string as str


def sql_palavras_chave(palavras):
    n = len(palavras)
    pch = ["''","''","''","''","''"]
    for i in range(0,5):
        if i<n:
            palavra = palavras[i]
            palavra = palavra.replace("'", "''")
            pch[i]=("'"+palavra+"'")
    return (",".join(pch))

def sql_key_words(palavras):
    n = len(palavras)
    pch = ["''","''","''","''","''"]
    for i in range(0,5):
        if i<n:
            palavra = palavras[i]
            palavra = palavra.replace("'", "''")
            pch[i]=("'"+palavra+"'")
    return (",".join(pch))

def read_xml_text(arq_xml):
    f=open(arq_xml,'r',encoding='latin1')
    t=f.read()
    f.close()
    return t


def query_insert_dados_artigo_xml(scielo_id,arq_xml):
    try:
        xml = sci.parse_xml(arq_xml)
        titulo = xml.get_title()['title']
        titulo = titulo.replace("'", "''")
        
        resumo = xml.get_resumo()['resumo']
        resumo = resumo.replace("'", "''")
        
        abstract = xml.get_abstract()['abstract']
        abstract = abstract.replace("'", "''")

        palavras_chave = xml.get_palavras_chave()['palavras_chave']

        palavras = sql_palavras_chave(palavras_chave)
        
        key_words = xml.get_keywords()['keywords']

        words = sql_key_words(key_words)
        
        texto_xml = xml.get_texto()
        soup = BeautifulSoup(texto_xml,"html5lib")
        texto = soup.get_text('\n')
        texto = texto.replace("'","''")
        
        
        values = f"""'{scielo_id}','{titulo}','{resumo}',{palavras},'{abstract}',{words},'{texto}'"""

        sql = f"""insert into alacip.artigos_dados_xml  
                (scielo_id,titulo,resumo,
                 palavra_chave1,
                 palavra_chave2,
                 palavra_chave3,
                 palavra_chave4,
                 palavra_chave5, 
                 abstract,
                 key_word1,
                 key_word2,
                 key_word3,
                 key_word4,
                 key_word5,
                 texto)
                 values ({values});
        """

    except (etree.XMLSyntaxError, etree.ParseError):
        sql = "insert into alacip.artigos_dados_xml values ("+"'"+scielo_id+"','XML Invalido','','','','');"
    return sql



### Obtêm o arquivo .xml de cada artigo selecionado e procede a inclusão na tabela dados_xml

In [ ]:

query_artigos = "select scielo_id from alacip.artigos_selecionados"

df = smod.pandas_query(query_artigos)

os.chdir(xml_dir)

for scielo_id in df['scielo_id']:
    xml_file = xml_dir+'/'+scielo_id +'.xml'
    file_path = Path(xml_file)

    query = query_insert_dados_artigo_xml(scielo_id,xml_file)

    smod.execute_query(query)

print("OK")



### Atualiza marcador de artigo empírico na tabela dados_xml

In [ ]:
query_update_tem_elemento_empirico = """
update alacip.artigos_dados_xml
set tem_tabela = '0',
    tem_quadro = '0',
    tem_grafico = '0',
    tem_diagrama = '0',
    tem_figura = '0',
    empirico = '0'
;

update alacip.artigos_dados_xml
set tem_tabela = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'tabela')
;

update alacip.artigos_dados_xml
set tem_tabela = '1'
WHERE to_tsvector('english', texto) @@ to_tsquery('english', 'table')
;

update alacip.artigos_dados_xml
set tem_grafico = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'grafico' || 'grafo')
;

update alacip.artigos_dados_xml
set tem_grafico = '1'
WHERE to_tsvector('english', texto) @@ to_tsquery('english', 'graphic' || 'plot' || 'graph')
;

update alacip.artigos_dados_xml
set tem_quadro = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'quadro')
;

update alacip.artigos_dados_xml
set tem_diagrama = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'diagrama')
;

update alacip.artigos_dados_xml
set tem_diagrama = '1'
WHERE to_tsvector('english', texto) @@ to_tsquery('english', 'diagram' || 'chart' )
;

update alacip.artigos_dados_xml
set tem_figura = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'figura')
;

update alacip.artigos_dados_xml
set empirico = '1' 
where tem_tabela = '1' or tem_quadro = '1' or tem_diagrama = '1' or tem_figura = '1'
;

"""

smod.execute_query(query_update_tem_elemento_empirico)

print("OK")


## Atualiza a tabela artigos selecionados com os dados XML

In [ ]:
query_update_artigos = """
with dados_xml as
(select 
    scielo_id  ,
    titulo,
    resumo,
    palavra_chave1  ,
    palavra_chave2  ,
    palavra_chave3  ,
    palavra_chave4  ,
    palavra_chave5  ,
    empirico  ,
    tem_tabela  ,
    tem_quadro  ,
    tem_grafico  ,
    tem_diagrama  ,
    tem_figura  ,
    abstract,
    key_word1  ,
    key_word2  ,
    key_word3  ,
    key_word4  ,
    key_word5  ,
    texto_xml 
 
from
    alacip.artigos_dados_xml
)

update alacip.artigos_selecionados as a
set
  resumo = x.resumo,
  palavra_chave1 = x.palavra_chave1 ,
  palavra_chave2 = x.palavra_chave2 ,
  palavra_chave3 = x.palavra_chave3 ,
  palavra_chave4 = x.palavra_chave4 ,
  palavra_chave5 = x.palavra_chave5 ,
  empirico = x.empirico ,
  tem_tabela = x.tem_tabela ,
  tem_quadro = x.tem_quadro ,
  tem_grafico = x.tem_grafico ,
  tem_diagrama = x.tem_diagrama ,
  tem_figura = x.tem_figura ,
  abstract = x.abstract ,
  key_word1 = x.key_word1 ,
  key_word2 = x.key_word2 ,
  key_word3 = x.key_word3 ,
  key_word4 = x.key_word4 ,
  key_word5 = x.key_word5 ,
  texto_xml = x.texto_xml
from 
    dados_xml as x
where 
    a.scielo_id like x.scielo_id
;
"""

smod.execute_query(query_update_artigos)